In [36]:
#!/usr/bin/env python
# coding: utf-8

##########################################################################################
#### Import libraries
import requests
import json
import os
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd
import glob
import numpy as np
import re
import shutil
import csv

In [37]:
print(os.getcwd())

df = pd.read_csv(
    "D:/devGit/CoastPilot/_deepzoom_database_output.csv",
    escapechar="\\")

d:\devGit\CoastPilot\Milestone1


In [38]:
df.head()

,source_id,feature_name,feature_class,lat_dec,long_dec,elev_in_m,elev_in_ft,paragraph
0,359188,Hilo Bay,Bay,19.7378,-155.0794,0.0,0.0,"<p class=""cp_paragraph""><span class=""cp_parain..."
1,1139611,Chetco Cove,Bay,42.0426,-124.2729,0.0,0.0,"<p class=""cp_paragraph""><span class=""cp_parain..."
2,1142784,Goat Island,Island,42.0657,-124.3226,37.0,121.39108,"<p class=""cp_paragraph""><span class=""cp_parain..."
3,1139346,Cape Ferrelo,Cape,42.1024,-124.3506,76.0,249.34384,"<p class=""cp_paragraph""><span class=""cp_parain..."
4,1134644,Yellow Rock,Island,42.0462,-124.2743,1.0,3.28084,"<p class=""cp_paragraph""><span class=""cp_parain..."


In [39]:
df.columns

Index(['source_id', 'feature_name', 'feature_class', 'lat_dec', 'long_dec',
       'elev_in_m', 'elev_in_ft', 'paragraph'],
      dtype='object')

In [40]:
features = ["Bay", "Canal", "Cape", "Channel", "River", "Stream"]
home = df[(df["lat_dec"] > 46) & (df["lat_dec"] < 48) & (df["long_dec"] < -121) & (df["long_dec"] > -124) & (df["feature_class"].isin(features)) ]


In [41]:
home.head()

,source_id,feature_name,feature_class,lat_dec,long_dec,elev_in_m,elev_in_ft,paragraph
144,1124623,Necanicum River,Stream,46.0115,-123.9274,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
152,1125500,Point Adams,Cape,46.2120,-123.9710,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
156,1533028,Baker Bay,Bay,46.2948,-123.9893,0.0,0.0,"<p class=""cp_paragraph""><span class=""cp_parain..."
162,1127030,Skipanon Waterway,Canal,46.1765,-123.9104,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
165,1127129,Smith Point,Cape,46.1832,-123.8615,3.0,9.84252,"<p class=""cp_paragraph""><span class=""cp_parain..."


In [43]:
len(home.index)

190

In [115]:
auth = requests.auth.HTTPBasicAuth('arnie', 'snarf')

url = "https://localhost:44346/api/v1/data/markersPublic"
#url = "https://deepzoom-test.azurewebsites.net/api/v1/data/markersPublic"
cer = "D:/devGit/CoastPilot/cert.pem"
key = "D:/devGit/CoastPilot/key.pem"

def upload(row):
    id = "GNIS" + str(row["source_id"])
    coordinates = [row["long_dec"], row["lat_dec"]]
    properties = {
        "isMarker": True,
        "access": "public",
        "name": row["feature_name"],
        "icon": "coastpilot",
        "id": id
    }
    doc = row["paragraph"]

    marker = {
        "id": id,
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": coordinates,
        },
        "properties": properties
    }
    payload = {
        "id": id,
        "point": marker,
        "doc": doc,
    }

    header = {"Content-type": "application/json"} 

    response = requests.post(url, data=payload, headers=header, auth=auth, cert=(cer, key))
    print (response.status_code, payload)
    # response_json = response_decoded_json.json()

In [114]:
for index, row in home.iterrows():
    #print(row['paragraph'])
    upload(row)
    if index > 3:
        break

SSLError: HTTPSConnectionPool(host='localhost', port=44346): Max retries exceeded with url: /api/v1/data/markersPublic (Caused by SSLError(SSLError(336265225, '[SSL] PEM lib (_ssl.c:3738)')))

In [45]:
home

,source_id,feature_name,feature_class,lat_dec,long_dec,elev_in_m,elev_in_ft,paragraph
144,1124623,Necanicum River,Stream,46.0115,-123.9274,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
152,1125500,Point Adams,Cape,46.2120,-123.9710,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
156,1533028,Baker Bay,Bay,46.2948,-123.9893,0.0,0.0,"<p class=""cp_paragraph""><span class=""cp_parain..."
162,1127030,Skipanon Waterway,Canal,46.1765,-123.9104,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
165,1127129,Smith Point,Cape,46.1832,-123.8615,3.0,9.84252,"<p class=""cp_paragraph""><span class=""cp_parain..."
167,1129394,Youngs Bay,Bay,46.1748,-123.8660,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
168,1129399,Youngs River,Stream,46.1809,-123.8751,2.0,6.56168,"<p class=""cp_paragraph""><span class=""cp_parain..."
169,1123054,Lewis and Clark River,Stream,46.1684,-123.8629,1.0,3.28084,"<p class=""cp_paragraph""><span class=""cp_parain..."
170,1507513,Point Ellice,Cape,46.2432,-123.8738,53.0,173.88452,"<p class=""cp_paragraph""><span class=""cp_parain..."
172,1128158,Tongue Point,Cape,46.2126,-123.7579,18.0,59.05512,"<p class=""cp_paragraph""><span class=""cp_parain..."
